# APS 4 
## * Deve trabalhar nesta APS com seu grupo do Projeto 1
## * Preencha os resultados desta APS no Blackboard
## * Entrega INDIVIDUAL HOJE até término desta aula

___
# Exercício: Camões&Lusíadas - Tomando decisão com Teorema de Bayes
___

## Aula 08

<div id="indice"></div>



## Índice

- [Análise "Os Lusíadas" vs "Dom Casmurro"](#analise-lusiadas-vs-dom-casmurro)
    - [Limpeza das bases](#limpeza-das-bases)
    - [Tabelas de frequências: Dom Casmurro](#tabelas-de-frequencias-dom-casmurro)
        - [Frequências absolutas](#frequencias-absolutas-casmurro)
        - [Frequências relativas](#frequencias-relativas-casmurro)
    - [Tabelas de frequências: Os Lusíadas](#tabelas-de-frequencias-os-lusiadas)
- [Probabilidades](#probabilidades)
- [Probabilidades na língua portuguesa](#probabilidades-na-lingua-portuguesa)
- [Classificação: Lusíadas ou Dom Casmurro?](#classificacao-lusiadas-ou-dom-casmurro)
    - [A ingenuidade - Naïve Bayes](#a-ingenuidade-naive-bayes)

- [O que você deve fazer](#o-que-voce-deve-fazer)
    - [Exercício 1](#ex1) **- APS4 disponível no Blackboard**
    - [Exercício 2](#ex2) **- APS4 disponível no Blackboard**
    - [Exercício 3](#ex3) **- APS4 disponível no Blackboard**
    - [Exercício 4](#ex4) **- APS4 disponível no Blackboard**
- [O que você deve pesquisar para fazer o Projeto 1](#o-que-pesquisar)

___
### Este notebook é uma prévia para o Projeto 1.

### A técnica apresentada neste notebook foi usada para resolver [um problema histórico de autoria de documentos](https://priceonomics.com/how-statistics-solved-a-175-year-old-mystery-about/).

### Leia [esta referência do Prof. Sebastian Raschka](https://arxiv.org/pdf/1410.5329.pdf) após concluir esta atividade. Também existe [em formato de blog](https://sebastianraschka.com/Articles/2014_naive_bayes_1.html)
___


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13

Abaixo definimos uma função de limpeza simples, que usaremos nos trechos de texto analizados

In [ ]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    

**Avaliando funcionamento da função `cleanup()`:**

In [ ]:
texto = f'A não ser que haja ! várias : pontuações ; excluir tudo ? não de novo A que haja haja haja'
texto

In [ ]:
len(texto.split())

In [ ]:
len(pd.Series(texto.split()))

In [ ]:
# Limpa texto mas não guarda limpeza
cleanup(texto)

In [ ]:
# Se quiser guardar texto limpo no mesmo objeto, então:
texto = cleanup(texto)
texto

<div id="analise-lusiadas-vs-dom-casmurro"></div>

## Análise "Os Lusíadas" vs "Dom Casmurro"

Vamos analisar as obras Dom Casmurro, de [Machado de Assis](http://machado.mec.gov.br/) e Os Lusíadas, de [Luis Vaz de Camões](https://pt.wikipedia.org/wiki/Lu%C3%ADs_de_Cam%C3%B5es). Ambas as obras são de domínio público e foram obtidas no site do [Projeto Gutenberg](https://www.gutenberg.org/wiki/PT_Principal).

Nosso objetivo é: dada uma frase extraída de alguma das duas obras, identificar a qual obra a frase pertence.
Ou seja, queremos classificar uma frase como pertencendo a "Os Lusíadas" ou a "Dom Casmurro". 

Para isso temos somente a informação de quais **palavras constituem a frase**. Como podemos utilizar as palavras para descobrir a qual obra a frase pertence? Você deve ter algum conhecido que sempre usa alguma palavra um pouco estranha ou diferente que somente ele/ela usa. A intuição no nosso caso é semelhante: cada autor deve ter algumas palavras específicas que usa com mais frequência do que o outro autor. Se isso for verdade, poderíamos tentar descobrir qual é a obra dependendo de quais palavras aparecem na frase.

Vamos pensar em termos de probabilidade. A informação que temos disponível sobre a frase são as palavras. Assim, temos os seguintes eventos:

- $Casmurro$: a frase foi extraída da obra "Dom Casmurro";
- $Lusíadas$: a frase foi extraída da obra "Os Lusíadas";
- $frase$: uma determinada frase foi encontrada.

Utilizamos $w_i$ como um nome genérico para qualquer palavra utilizada em algum dos textos.

A partir das palavras contidas na frase, queremos calcular a probabilidade da frase ter sido extraída de "Dom Casmurro" ou de "Os Lusíadas". Traduzindo em termos de probabilidades:

- $P(Casmurro|frase)$: probabilidade da obra original ser "Dom Casmurro" dado a frase;
- $P(Lusíadas|frase)$: probabilidade da obra original ser "Os Lusíadas" dado a frase.

Se $P(Casmurro|frase) > P(Lusíadas|frase)$ classificaremos a frase como extraída de "Dom Casmurro", senão, classificaremos como "Os Lusíadas". Mas como podemos calcular essas probabilidades?

Vimos na atividade da aula (dataset do Titanic), que é possível utilizar o **Teorema de Bayes** para obter uma probabilidade condicional $P(Casmurro|frase)$ da seguinta forma:

$$P(Casmurro|frase) = \frac{P(frase|Casmurro) P(Casmurro)}{P(frase)}$$

<br>

<br>

O processo é análogo para $P(Lusíadas|frase)$:

$$P(Lusíadas|frase) = \frac{P(frase|Lusíadas) P(Lusíadas)}{P(frase)}$$

Ou seja, precisamos de: 
- $P(frase|Casmurro)$: probabilidade de encontrar a frase na obra "Dom Casmurro";
- $P(frase|Lusíadas)$: probabilidade de encontrar a frase na obra "Os Lusíadas";
- $P(Casmurro)$: probabilidade da obra original ser "Dom Casmurro";
- $P(Lusíadas)$: probabilidade da obra original ser "Os Lusíadas";
- $P(frase)$: probabilidade de uma frase ocorrer na língua portuguesa.

Veremos que temos como conseguir estimativas de todas essas probabilidades. Assim poderemos obter as probabilidades desejadas. 

**Observação**: classificadores como o que vamos construir, que utilizam a regra de Bayes para calcular as probabilidades, são chamados Classificadores Bayesianos. Na verdade, vamos utilizar uma simplificação, conhecida como **Naïve Bayes**.

Vamos começar carregando o texto de cada obra.

In [ ]:
import os
print(f'Esperamos trabalhar no diretório: \n{os.getcwd()}\n')

In [ ]:
os.listdir('textos')

In [ ]:
filename = 'lusiadas_texto.txt'

if filename in os.listdir('textos'):
    print(f'Parece que o arquivo {filename} está na pasta diretório correto, yay!')
    
else:
    print(f'Não encontrei o arquivo {filename}')

In [ ]:
filename = 'domcasmurro_texto.txt'

if filename in os.listdir('textos'):
    print(f'Parece que o arquivo {filename} está na pasta diretório correto, yay!')
    
else:
    print(f'Não encontrei o arquivo {filename}')

In [ ]:
# Carrega o arquivo de texto
# Como ele possui caracteres com acentos, precisamos carregar utilizando o encoding="utf8".
with open("textos/lusiadas_texto.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    lusiadas_raw = arquivo_texto.read()

# Imprime os 200 primeiros caracteres do texto para verificar se está tudo ok.
print(lusiadas_raw[0:200])

In [ ]:
with open("textos/domcasmurro_texto.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    casmurro_raw = arquivo_texto.read()

# Imprime os 200 primeiros caracteres do texto para verificar se está tudo ok.
print(casmurro_raw[0:200])

<div id="limpeza-das-bases"></div>

### Limpezas das bases

Em nossa análise será necessário calcular a frequência das palavras no texto. Entretanto, o Python diferencia caracteres maiúsculos (e.g. `'A'` ou `'B'`) de minúsculos (e.g. `'a'` e `'b'`). Por isso, vamos converter todo o texto para letras minúsculas com `.lower()`.

Além disso, apesar de serem importantes para a língua, vamos simplificar nossa análise removendo sinais de pontuação. Para isso vamos aplicar uma limpeza rudimentar com a função `cleanup()` definida anteriormente.

In [ ]:
lusiadas = cleanup(lusiadas_raw.lower())

In [ ]:
casmurro = cleanup(casmurro_raw.lower())

Vamos inspecionar  os arquivos após limpeza visualizando os 200 primeiros caracteres

In [ ]:
print(lusiadas[0:200])

In [ ]:
print(casmurro[0:200])

<div id="tabelas-de-frequencias-dom-casmurro"></div>

### Tabelas de frequências: Dom Casmurro

Primeiro, vamos quebrar o texto em palavras utilizando o comando `.split()`.

In [ ]:
# Verificando o uso do split para as primeiras 10 palavras
casmurro.split()[0:10]

Nosso objetivo é contar as palavras de cada texto, ou seja, construir a tabela de frequência absoluta para as palavras de cada obra. 

Assim, vamos converter as obras (strings contendo o texto completo) em objetos do tipo `pd.Series`, nos quais cada elemento é uma palavra. 

In [ ]:
# Quebrando o texto em palavras
todas_palavras_casmurro = casmurro.split()

# Guardando as palavras como um pd.Series
serie_casmurro = pd.Series(todas_palavras_casmurro)
serie_casmurro.head(10)

<div id="frequencias-absolutas-casmurro"></div>

#### Frequências absolutas

Palavras em um texto são variáveis **qualitativas nominais**, portanto usaremos `value_counts()` para obter a tabela de frequências relativas e absolutas:

In [ ]:
# Como serie_casmurro é uma pd.Series, podemos usar value_counts() nessa variável
tabela_casmurro = serie_casmurro.value_counts()
tabela_casmurro

<div id="frequencias-relativas-casmurro"></div>

#### Frequências relativas

In [ ]:
tabela_casmurro_relativa = serie_casmurro.value_counts(True)
tabela_casmurro_relativa

É de se estranhar a linha abaixo?

In [ ]:
tabela_casmurro_relativa.sum()


Agora faremos exatamente os mesmos passos de converter em `Series` e obter as tabelas de frequência para a obra "Os Lusíadas"

<div id="tabelas-de-frequencias-os-lusiadas"></div>

### Tabelas de frequências: "Os Lusíadas"

In [ ]:
# Verificando o uso do split para as primeiras 10 palavras
lusiadas.split()[0:10]

In [ ]:
# Quebrando o texto em palavras
todas_palavras_lusiadas = lusiadas.split()

# Guardando as palavras como um pd.Series
serie_lusiadas = pd.Series(todas_palavras_lusiadas)
serie_lusiadas.head(10)

<div id="frequencias-absolutas-lusiadas"></div>

#### Frequências absolutas

In [ ]:
# Não funciona porque é lista e não pd.Series. Comando value_counts() funciona com pd.Sereis
#todas_palavras_lusiadas.value_counts() 

In [ ]:
# Aqui a lista foi redefinida como pd.Series(). Então comando value_counts() funciona.
#serie_lusiadas.value_counts() 

In [ ]:
tabela_lusiadas = serie_lusiadas.value_counts()
tabela_lusiadas

<div id="frequencias-relativas-lusiadas"></div>

#### Frequências relativas

In [ ]:
tabela_lusiadas_relativa = serie_lusiadas.value_counts(True)
tabela_lusiadas_relativa

<div id="probabilidades"></div>

## Probabilidades

Relembrando, queremos descobrir, dentre outras coisas, a probabilidade de uma frase ocorrer em uma das obras. Para isso, vamos assumir uma simplificação **bastante ingênua** de que a probabilidade de uma palavra da frase ocorrer em uma obra é independente da probabilidade de qualquer outra palavra. Assim, assumindo independência, **a probabilidade da frase é igual ao produto das probabilidades de cada palavra individualmente**.

Vamos assumir para fins destas análises que **a frequência relativa observada nestes textos é igual à probabilidade**. Ou seja, que se quisermos encontrar $P(portugal|Lusíadas)$ basta checar a frequência da palavra `'portugal'`: `tabela_lusiadas_relativa["portugal"]` (note que o acesso é semelhante ao acesso a um dicionário).

Ou seja, a $P(portugal|Lusíadas)$  pode ser obtida buscando a frequência relativa da palavra `portugal` dentro da tabela do texto Os Lusíadas:

In [ ]:
tabela_lusiadas_relativa["portugal"]

Da mesma fora, se quisermos $P(capitú|casmurro)$ basta checar: 

In [ ]:
tabela_casmurro_relativa["capitú"]

Notamos que na versão do *Gutenberg* a palavra *Capitú* aparece grafada com acento.

<div id="probabilidades-na-lingua-portuguesa"></div>

## Probabilidades na língua portuguesa

Vamos fazer mais uma simplificação. Vamos assumir que o todo da língua portuguesa fosse formado pela fusão das obras *Dom Casmurro* e *Os Lusíadas*.

Nesse caso, vamos juntar os dois textos já limpos pela função `cleanup` que foram guardados em  `lusiadas` e `casmurro`

In [ ]:
portugues = lusiadas + casmurro

Vamos repetir os passos anteriores quebrando o texto guardando em `portugues` em palavras e depois guardá-lo em uma `pd.Series()`.

In [ ]:
todas_palavras_portugues = portugues.split()
serie_portugues = pd.Series(todas_palavras_portugues)

Podemos refazer a análise de probabilidades considerando este novo *corpus* de texto que contem as duas obras:

In [ ]:
tabela_portugues_relativa = serie_portugues.value_counts(True)
tabela_portugues_relativa

Desta forma, se quisermos saber a probabilidade da palavra *pintura* em toda a "língua portuguesa", a notação seria simplesmente $P(pintura)$ porque estamos a assumir que trabalhamos com a totalidade da língua, ou seja **o conjunto universo**.

Assim, a probabilidade $P(pintura)$ é:

In [ ]:
tabela_portugues_relativa["pintura"]

<div id="classificacao-lusiadas-ou-dom-casmurro"></div>

## Classificação de uma frase: Lusíadas ou Dom Casmurro?

Agora vamos ao problema que queremos resolver.

Você precisa dizer se a **frase**: 

$\quad$ **"Contou que João como santo se verá vestido, de maneira  que virão a barba do marido"**

é mais provável de ter vindo de *Os Lusíadas* ou de *Dom Casmurro*.

Ou seja, precisa decidir: 

$\quad \Rightarrow$ Se $P(Casmurro|frase) > P(Lusíadas|frase)$, então frase será classificada como de *Dom Casmurro*

$\quad \Rightarrow$ Se $P(Casmurro|frase) < P(Lusíadas|frase)$, então frase será classificada como de *Os Lusíadas*


<br>

A partir deste ponto, vamos denotar $Casmurro$ como $C$ e $Lusíadas$ como $L$ para brevidade.

O **Teorema de Bayes** vai ser particularmente útil neste caso. Lembre-se que:

$\quad P(C|frase) = \frac{P(frase|C)P(C)}{P(frase)}$

e que:

$\quad P(L|frase) = \frac{P(frase|L)P(L)}{P(frase)}$


<br>
Teremos que calcular **TODAS** as probabilidades que se encontram no lado direito das equações acima e com elas calcular $P(C|frase)$ e $P(L|frase)$.


### Frase a classificar:

In [ ]:
frase = "Contou que João como santo se verá vestido, de maneira  que virão a barba do marido"

Todo o preparo que as obras passaram, devem se repetir na frase.

Vamos converter primeiro em minúsculas e fazer a limpeza:

In [ ]:
frase = cleanup(frase.lower())

Agora a frase está assim:

In [ ]:
frase

Em forma de lista, para facilitar o processamento:

In [ ]:
frase.split()

<div id="a-ingenuidade-naive-bayes"></div>

### A ingenuidade - Naïve Bayes

Agora vamos à parte ingênua do Naïve Bayes, que consiste em assumir que as palavras são independentes entre si e que sua ordem na frase não importa. 


Ou seja:

$\quad P(frase|C) = 
P(contou|C)\cdot P(que|C)\cdot P(joão|C)\cdot P(como|C)\cdot P(santo|C)\cdot P(se|C)\cdot P(verá|C)\cdot P(vestido|C)\cdot P(de|C)\cdot...$
$\qquad\qquad\qquad\quad ...\cdot P(maneira|C)\cdot P(que|C)\cdot P(virão|C)\cdot P(a|C)\cdot P(barba|C)\cdot P(do|C)\cdot P(marido|C)$

<br>

Vamos denotar **Dom Casmurro** simplesmente como $C$, para encurtar a fórmula a seguir aplicando o uso do Teorema de Bayes.

Assim, a fórmula completa fica:

$P(C|frase) = \frac{P(contou|C).P(que|C).P(joão|C).P(como|C).P(santo|C).P(se|C).P(verá|C).P(vestido|C).P(de|C).P(maneira|C).P(que|C).P(virão|C).P(a|C).P(barba|C).P(do|C).P(marido|C).P(C)}{P(frase)}$

<br>

Da mesma forma, denotando **Os Lusíadas** como $L$, a fórmula completa fica:

$P(L|frase) = \frac{P(contou|L).P(que|L).P(joão|L).P(como|L).P(santo|L).P(se|L).P(verá|L).P(vestido|L).P(de|L).P(maneira|L).P(que|L).P(virão|L).P(a|L).P(barba|L).P(do|L).P(marido|L).P(L)}{P(frase)}$

<br>

Note que o denominador das duas probabilidade condicionais acima é o mesmo, para fazer a classificação da frase podemos cancelar o cálculo do denominador $P(frase)$. 

Assim, a **Classificação da frase** se dará conforme abaixo:

$\quad \Rightarrow$ Se $P(C|frase) > P(L|frase)$, então frase será classificada como de *Dom Casmurro*

$\quad \Rightarrow$ Se $P(C|frase) < P(L|frase)$, então frase será classificada como de *Os Lusíadas*




<div id="o-que-voce-deve-fazer"></div>

# [TAREFA] Faça os cálculos e classifique a seguinte frase:

In [ ]:
frase

<div id="ex1"></div>

### EXERCÍCIO 1

Como podemos calcular os valores dos priors $P(C)$ e $P(L)$ ? Ou, colocando a pergunta em termos Bayesianos: o que é um *prior* razoável para se usar?

<br> 
**Dica**:

Você pode usar proporção de número de palavras do conjunto *Lusíadas* e *Dom Casmurro* em relação ao total. Ou ainda proporção destas palavras ponderadas pela frequência absoluta.

<br>

Armazene o valor de $P(C)$ e $P(L)$ nas variáveis `probC` e `probL`, respectivamente.

<br>

In [ ]:
# ESCREVA SEU CÓDIGO AQUI


<div id="ex2"></div>

### EXERCÍCIO 2

Calcule os termos: 

$\quad P(frase|L) = 
P(contou|L)\cdot P(que|L)\cdot P(joão|L)\cdot P(como|L)\cdot P(santo|L)\cdot P(se|L)\cdot P(verá|L)\cdot P(vestido|L)\cdot P(de|L)\cdot...$
$\qquad\qquad\qquad\quad ...\cdot P(maneira|L)\cdot P(que|L)\cdot P(virão|L)\cdot P(a|L)\cdot P(barba|L)\cdot P(do|L)\cdot P(marido|L)$

e 

$\quad P(frase|C) = 
P(contou|C)\cdot P(que|C)\cdot P(joão|C)\cdot P(como|C)\cdot P(santo|C)\cdot P(se|C)\cdot P(verá|C)\cdot P(vestido|C)\cdot P(de|C)\cdot...$
$\qquad\qquad\qquad\quad ...\cdot P(maneira|C)\cdot P(que|C)\cdot P(virão|C)\cdot P(a|C)\cdot P(barba|C)\cdot P(do|C)\cdot P(marido|C)$

<br>

Armazene o valor de $P(frase|L)$ e $P(frase|C)$ nas variáveis `probFraseDadoC` e `probFraseDadoL`.

<br>


In [ ]:
# ESCREVA SEU CÓDIGO AQUI

<div id="ex3"></div>

### EXERCÍCIO 3

Calcule $P(L|frase)$ e $P(C|frase)$ como indicado acima. 

Lembre-se que não precisamos descobrir o valor de $P(frase)$, pois ele é denominador comum de ambos os lados da comparação. 

<br>


In [ ]:
# ESCREVA SEU CÓDIGO AQUI

<div id="ex4"></div>

### EXERCÍCIO 4

Escreva seu parecer: você acha que o texto indicado pertence a Os Lusíadas ou a Dom Casmurro?

<br>

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

ESCREVA SUA RESPOSTA AQUI

<div id="o-que-pesquisar"></div>

# O que você deve pesquisar para fazer o Projeto 1:

No [texto recomendado](https://arxiv.org/pdf/1410.5329.pdf) estude o que fazer quando duas situações acontecerem:
- Quando aparecem palavras inéditas para classificar
- Quando a multiplicação das probabilidades é um valor tão pequeno que ocorre *underflow*